In [57]:
import asyncio
from pyppeteer import launch
import csv
import pandas as pd
import re
from datetime import date
import os
from google.cloud import bigquery

In [ ]:
job_df=None

In [21]:
####test fields
async def scrape_indeed():
    global job_df
    global job_listings
    browser = await launch(headless=False)
    page = await browser.newPage()


    await page.goto('https://www.indeed.com')

    # Wait for 5 seconds
    await asyncio.sleep(5) 
    # Input job title
    await page.type('input[name="q"]', "data analyst")

     # Clear the default location
    location_input_selector = 'input[name="l"]'
    await page.click(location_input_selector, {'clickCount': 3})  # Triple click to select all text
    await page.keyboard.press('Backspace')  # Clear the text

    # Input new location
    await page.type(location_input_selector, "Dallas, TX")

    # Click search button
    await page.click('button[type="submit"]')
    # Wait for 5 seconds
    await asyncio.sleep(5) 


    # Apply "Date Posted" filter for "Last 3 days"
    await page.waitForSelector('#filter-dateposted', {'visible': True})
    await page.click('#filter-dateposted')

    # Wait for the dropdown to become visible and select the "Last 3 days" option
    await page.waitForSelector('a[href*="&fromage=3"]', {'visible': True})
    await page.click('a[href*="&fromage=3"]')
    await asyncio.sleep(3) 
    #select 100 miles radius
    await page.waitForSelector('#filter-radius',{'visible':True})
    await page.click('#filter-radius')
    await page.waitForSelector('a[href*="&radius=100"]',{'visible':True})
    await page.click('a[href*="&radius=100"]')


    # Max go to 10 pages
    num_pages=10
    job_info={
        'title':[],
        'company':[],
        'location':[],
        'post_date': [],
        'rating':[],
        'salary':[],
        'job_link':[],
        'jd':[]
    }

    for pg in range(num_pages):


            

        # Wait for 5 seconds
        await asyncio.sleep(5) 

        job_listings = await page.querySelectorAll('.job_seen_beacon')
        
        for job in job_listings:
            # Extract the job title
            try:
                title_element = await job.querySelector('h2.jobTitle span[title]')
                title = await page.evaluate('(element) => element.textContent', title_element)
            except Exception as e:
                title="Not Found"


            # Extract the company name
            try:
                company_element = await job.querySelector('div.company_location [data-testid="company-name"]')
                company = await page.evaluate('(element) => element.textContent', company_element)
            except Exception as e:
                company="Not Found"

            # Extract jd
            try:
                jd_element = await job.querySelector('div.underShelfFooter div ul')
                jd = await page.evaluate('(element) => element.textContent', jd_element)
            except Exception as e:
                jd="Not Found"

            # Get job link
            try:
                job_element = await job.querySelector('h2.jobTitle a')
                job_link = await page.evaluate('(element) => element.getAttribute("href")', job_element)
            except Exception as e:
                job_link="Not Found"


            # Extract the rating
            try:
                rating_element = await job.querySelector('div.company_location span [data-testid="holistic-rating"]')
                rating = await page.evaluate('(element) => element.textContent', rating_element)
            except Exception as e:
                rating="Not Found"

            # Extract the date
            try:
                date_element = await job.querySelector('div.underShelfFooter [data-testid="myJobsStateDate"]')
                post_date = await page.evaluate('(element) => element.textContent', date_element)
            except Exception as e:
                post_date="Not Found"

            # Extract the salary
            try:
                salary_element = await job.querySelector('div.jobMetaDataGroup [data-testid="attribute_snippet_testid"]')
                salary = await page.evaluate('(element) => element.textContent', salary_element)
            except Exception as e:
                salary="Not Found"

            # Extract the location
            location_element = await job.querySelector('div.company_location [data-testid="text-location"]')
            location = await page.evaluate('(element) => element.textContent', location_element)

            job_info['title'].append(title)
            job_info['company'].append(company)
            job_info['location'].append(location)
            job_info['post_date'].append(post_date)
            job_info['rating'].append(rating)
            job_info['salary'].append(salary)
            job_info['job_link'].append(job_link)
            job_info['jd'].append(jd)

            #click for next page
        try:
            await page.waitForSelector('nav [data-testid="pagination-page-next"]')
            await page.click('nav [data-testid="pagination-page-next"]')
        except Exception as e:
            print("no more page")
            break
        
        

    job_df=pd.DataFrame.from_dict(job_info)

    await browser.close()

In [22]:
await scrape_indeed()

no more page


In [56]:
job_df

,title,company,location,post_date,rating,salary,job_link,jd,pay_period,salary_first,salary_second,data_date
0,"Department Data Analyst, HRPP",UT Southwestern,"Dallas, TX 75390 (Design District area)",Just posted,Not Found,Full-time,/rc/clk?jk=e12fd18049322537&bb=vz9NTSO-L8FBnh1...,"\n Abstracts data, reconciles data inconsiste...",None,NaN,NaN,2024-06-25
1,Data Infrastructure Analyst,"7-Eleven, Inc.","Irving, TX 75063 (Valley Ranch area)",3 days ago,Not Found,Full-time,/rc/clk?jk=2afdda60528682db&bb=vz9NTSO-L8FBnh1...,\n Manage processes and improve data set-up i...,None,NaN,NaN,2024-06-25
2,Health and Welfare Business Analyst,Fidelity Investments,"Westlake, TX 76262",Just posted,Not Found,Not Found,/rc/clk?jk=4e3043ddd8cc9cfa&bb=vz9NTSO-L8FBnh1...,\n Requirements Analysts will be charged with...,None,NaN,NaN,2024-06-25
3,Business Analyst,McKesson,"Irving, TX 75039 (Freeport/Hackberry area)",Just posted,Not Found,"$59,700 - $99,500 a year",/rc/clk?jk=36baed663181cb0f&bb=vz9NTSO-L8FBnh1...,\n 2 + years business analyst experience.\n E...,yearly,59700.0,99500.00,2024-06-25
4,"Sr Analyst, Inflight Analytics",American Airlines,"Dallas, TX 75219 (Downtown area)",1 day ago,Not Found,Not Found,/rc/clk?jk=f9e9ef65e8e797ed&bb=vz9NTSO-L8FBnh1...,\n Strong analytical background involving dat...,None,NaN,NaN,2024-06-25
5,CRM Data Analysts – Lead,InfosysLimited,"Dallas, TX 75202 (Downtown area)",Just posted,Not Found,"$90,000 - $100,000 a year",/rc/clk?jk=b894b55a10273b0a&bb=vz9NTSO-L8FBnh1...,"\n Expert experience with data ETL process, d...",yearly,90000.0,100000.00,2024-06-25
6,"AVP, Analytics",Bank OZK,"Dallas, TX",Just posted,Not Found,Full-time,/rc/clk?jk=a350db4f51b96691&bb=vz9NTSO-L8FBnh1...,\n Minimum of three (3) years’ experience dev...,None,NaN,NaN,2024-06-25
7,Senior Data & Insights Analyst- 24010451,MoneyGram,"Remote in Dallas, TX",Just posted,Not Found,Not Found,/rc/clk?jk=6b0136b7986d482e&bb=vz9NTSO-L8FBnh1...,\n Proficient in SQL to query large data plat...,None,NaN,NaN,2024-06-25
8,"Business Analyst, Payments/Marketplace",Wellfit Technologies Inc.,"Hybrid work in Irving, TX 75039",Just posted,Not Found,"From $120,000 a year",/rc/clk?jk=db0613b310994fee&bb=vz9NTSO-L8FBnh1...,\n Full-cycle business analyst lifecycle: 5 y...,yearly,120000.0,NaN,2024-06-25
9,Business Analyst (Retail Banking),Wipro Limited,"Irving, TX",Just posted,Not Found,Not Found,/rc/clk?jk=95a502b8e88d52e6&bb=vz9NTSO-L8FBnh1...,\n You are expected to elicit and detail the ...,None,NaN,NaN,2024-06-25


In [24]:
#function to parse salary range
def salary_limit(salary):
    matches=re.findall(r'\$([\d,]+(\.\d+)?)',salary)
    first = matches[0][0] if len(matches)>0 else None
    second = matches[1][0] if len(matches)>1 else None
    return first,second
#function to parse salary pay period
def salary_period(salary):
    if salary.endswith('hour'):
        return 'hourly'
    elif salary.endswith('month'):
        return 'monthly'
    elif salary.endswith('year'):
        return 'yearly'


In [25]:
job_df['pay_period']=job_df['salary'].apply(salary_period)

In [26]:
job_df[['salary_first','salary_second']]=job_df['salary'].apply(lambda x: pd.Series(salary_limit(x)))

In [27]:
job_df['salary_first']=job_df['salary_first'].str.replace(',','').astype(float)
job_df['salary_second']=job_df['salary_second'].str.replace(',','').astype(float)

In [39]:
job_df['post_date']=job_df['post_date'].str.replace("Posted","").str.replace("Employer","")


In [47]:
job_df['data_date']=date.today()

In [53]:
# connect to Bigquery
os.environ['GOOGLE_APPLICATION_CREDENTIALS']="/Users/haobomini/Documents/bigquery/job-searching-427504-ddb233f06575.json"

In [62]:
client=bigquery.Client()
table_id='job-searching-427504.job_scrape.indeed_jobs'
info=client.load_table_from_dataframe(job_df,table_id)

encounter errors


print(error)